In [82]:
import pandas as pd
df = pd.read_csv("C:/Users/dell/Downloads/ccx_row_input_data.csv",encoding='latin-1')

In [83]:
df.head()

,profileId,programId,score,name,showType,language,languageShort,originalAssetId,source
0,[ee535a1d-6034-4bd9-a154-b21dd7a8e03c],667969ThirdParty3331353638303536,30.00,L'enlÌ¬vement 2,Movie,fr,fr,ASTR1000000700030953,assetDetails
1,[df82733a-97c1-4002-81fa-fe6933600eb3],667969ThirdParty3332393432343436,30.00,ÌÛ la valdrague,TVShow,fr,fr,DELP0020000000890200,assetDetails
2,[15221437-5617-4bba-a57e-ab7febed831e],667969ThirdParty373732323236,30.00,Taking Chance,Movie,en,en,CPHD2243773000000001,assetDetails
3,[c6f96fd7-a955-4fb2-9446-06be4d80e09a],667969ThirdParty3330313932323231,10.64,Rapides et dangereux,Movie,fr,fr,AAAA0000000016209900,assetDetails
4,[ea34f2aa-0baa-49e3-b7a7-a7873b2b8db6],667969ThirdParty3330313932323231,35.48,Rapides et dangereux,Movie,fr,fr,AAAA0000000016209900,assetDetails


In [84]:
#ScoreRules(Map(favorite -> 60, details-view -> 10, asset-purchase -> 100, rented -> 90, 
#un-favorite -> 0, vod-playback-completed -> 90, vod-playback-start -> 30))

# ALS MODEL,KNN,SVD, NEUTRAL and the accuracy of model and give recommendation to each user

In [85]:
df.showType.value_counts()

Movie     6055
TVShow    3621
Name: showType, dtype: int64

In [86]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit # The Cython library

In [118]:
df = df[['profileId', 'programId', 'score', 'name']]

In [119]:
data = df.dropna()

In [120]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [121]:
data['profileId'] = data['profileId'].astype("category")
data['profileId'] = le.fit_transform(data['profileId'])

In [122]:
data['programId'] = data['programId'].astype("category")
le_1 = LabelEncoder()
data['programId'] = le_1.fit_transform(data['programId'])

In [123]:
data['name'] = data['name'].astype("category")
le_2 = LabelEncoder()
data['name'] = le_2.fit_transform(data['name'])


In [124]:
sparse_item_user = sparse.csr_matrix((data['name'].astype(float), (data['profileId'], data['programId'])))
sparse_user_item = sparse.csr_matrix((data['name'].astype(float), (data['programId'], data['profileId'])))


In [125]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)


alpha_val = 40
data_conf = (sparse_item_user * alpha_val).astype('double')


model.fit(data_conf)

In [126]:
user_id =   14
recommended = model.recommend(user_id, sparse_user_item)
print(recommended)

[(160, 0.85518754), (116, 0.8532704), (405, 0.8479298), (1, 0.83914936), (627, 0.82325965), (31, 0.82067764), (307, 0.8188943), (624, 0.8067285), (309, 0.8063899), (662, 0.79505545)]


In [127]:
#Get similar items
item_id = 7
n_similar = 3
similar = model.similar_items(item_id, n_similar)
print(similar)


[(7, 0.02331074), (606, 0.016122831), (605, 0.016004242)]


In [128]:
model.fit(sparse_item_user)

In [146]:
model.recommend(7,sparse_user_item,N=5)

[(667, 0.42453563),
 (309, 0.39631045),
 (65, 0.38174456),
 (426, 0.36204422),
 (266, 0.3428424)]

In [143]:
df['name'][df.programId==le_1.inverse_transform([453]).item()].values[0]

'TripTank'